- https://github.com/PSY222/Colorinsight  git clone
- git에 있는 pth를 다운받음.



In [ ]:
!git clone https://github.com/PSY222/Colorinsight.git ##수정

Cloning into 'Colorinsight'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 86 (delta 6), reused 5 (delta 0), pack-reused 57 (from 1)
Receiving objects: 100% (86/86), 109.40 MiB | 21.05 MiB/s, done.
Resolving deltas: 100% (8/8), done.


- pth를 pt로 만들어서 매번 모델 로드 하지 않게 함.
- pt가 있으면 실행ㄴㄴ

In [4]:
import torch
import torchvision.models as models
import torch.nn as nn

# 모델 정의
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 4)

# state_dict 불러오기
model.load_state_dict(torch.load('best_model_resnet_ALL.pth', map_location='cpu'))

# 모델 전체 저장
torch.save(model, 'best_model_resnet_ALL.pt')


- best_model_resnet_ALL.pt를 이용해서 퍼스널 컬러 진단.
- pt 다운받아서 아래 코드만 실행.

In [ ]:
# pip install torch torchvision

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


사진 1개만 쓸 때

In [8]:
import torch
import torchvision.transforms as transforms
from PIL import Image

def get_season(img_path):
    # 모델 전체 로드 (구조 + 가중치)
    model = torch.load('best_model_resnet_ALL.pt', map_location='cpu', weights_only=False)
    model.eval()

    # 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # 이미지 로드 및 변환
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0)

    # 예측
    with torch.no_grad():
        output = model(image)

    pred_index = output.argmax().item()
    classes = ['봄웜', '여름쿨', '가을웜', '겨울쿨']
    print(f"Decided color: {classes[pred_index]} ({pred_index})")
    return pred_index

# 예시 실행
get_season("C:/Users/user/Desktop/test5.jpg")


Decided color: 겨울쿨 (3)


3

사진 여러 개

In [3]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from collections import Counter

# 클래스 이름
CLASSES = ['봄웜', '여름쿨', '가을웜', '겨울쿨']

# 이미지 전처리 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 모델 불러오기
def load_model(model_path='best_model_resnet_ALL.pt'):
    model = torch.load(model_path, map_location='cpu', weights_only=False)
    model.eval()
    return model

# 단일 이미지 예측 함수
def predict_single(model, img_path):
    image = Image.open(img_path).convert('RGB')
    image = transform(image).unsqueeze(0)
    with torch.no_grad():
        output = model(image)
    return output

# 방식 1: Hard Voting (가장 많이 나온 클래스)
def get_season_hard_vote(img_paths):
    model = load_model()
    results = []
    for path in img_paths:
        output = predict_single(model, path)
        pred = output.argmax().item()
        results.append(pred)

    most_common = Counter(results).most_common(1)[0][0]
    print(f"✅ Hard Voting 결과: {CLASSES[most_common]} (index {most_common})")
    return most_common

# 방식 2: Soft Voting (확률 평균)
def get_season_soft_vote(img_paths):
    model = load_model()
    all_probs = []
    for path in img_paths:
        output = predict_single(model, path)
        probs = F.softmax(output, dim=1).numpy()
        all_probs.append(probs)

    mean_prob = np.mean(np.vstack(all_probs), axis=0)
    pred_index = np.argmax(mean_prob)
    print(f"✅ Soft Voting 결과: {CLASSES[pred_index]} (index {pred_index})")
    print(f"Confidence scores: {mean_prob}")
    return pred_index

# 테스트 예시
if __name__ == "__main__":
    img_list = [
        "C:/Users/user/Desktop/test.jpg",
        "C:/Users/user/Desktop/test2.jpg",
        "C:/Users/user/Desktop/test3.jpg",
        "C:/Users/user/Desktop/test4.jpg",
        "C:/Users/user/Desktop/test5.jpg"
    ]
    
    print("\n🔸 Hard Voting 방식:")
    get_season_hard_vote(img_list)

    print("\n🔸 Soft Voting 방식:")
    get_season_soft_vote(img_list)



🔸 Hard Voting 방식:
✅ Hard Voting 결과: 여름쿨 (index 1)

🔸 Soft Voting 방식:
✅ Soft Voting 결과: 여름쿨 (index 1)
Confidence scores: [0.04919267 0.5238123  0.1427133  0.28428173]
